In [19]:
import pandas as pd
import numpy as np
import os

In [20]:
dir = ""

In [3]:
league_df = pd.read_csv(dir + "league.csv")

In [4]:
player_by_match_df = pd.read_csv(dir + "player_attributes_by_match.csv")

player_by_match_df = player_by_match_df.drop('Unnamed: 0',axis=1)

player_by_match_df = player_by_match_df.drop(["preferred_foot", "day_diff"], axis=1)

In [5]:
player_by_match_df.columns

Index(['date', 'match_id', 'player_id', 'overall_rating', 'potential',
       'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'dribbling', 'free_kick_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'reactions', 'shot_power', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'penalties', 'marking', 'standing_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'attacking_work_rate_imp', 'defensive_work_rate_imp', 'volleys_imp',
       'curve_imp', 'agility_imp', 'balance_imp', 'jumping_imp', 'vision_imp',
       'sliding_tackle_imp', 'team_id'],
      dtype='object')

In [6]:
match_df = pd.read_csv(dir + "match_clean.csv")

match_df = match_df.drop('Unnamed: 0',axis=1)

In [7]:
player_by_match_df.groupby(["match_id", "player_id"])["team_id"].nunique().value_counts()

1    541991
2        13
Name: team_id, dtype: int64

# Odds by match

In [8]:
odds_col = match_df.columns[32:63].tolist()

odds_df = match_df[["date", "match_id", "home_team_id", "away_team_id"]]

for i in range(0, len(odds_col), 3):
    print(odds_col[i:i+3])
    
    cur_odds_df = match_df.loc[~match_df[odds_col[i]].isnull(), 
                               ["date", "match_id", "home_team_id"] + odds_col[i:i+3]]
    
    inv_odds = (1/cur_odds_df[odds_col[i:i+3]])

    booksum = inv_odds.sum(axis=1)

    margin = booksum - 1

    inv_odds["booksum"] = booksum

    bookprob = pd.concat([inv_odds[odds_col[i]]/inv_odds["booksum"], 
                          inv_odds[odds_col[i+1]]/inv_odds["booksum"], 
                          inv_odds[odds_col[i+2]]/inv_odds["booksum"]], axis=1)

    bookprob.columns = odds_col[i:i+3]
    
    bookprob["{}_margin".format(odds_col[i][:-1])] = margin
    
    bookprob[["date", "match_id", "home_team_id"]] = cur_odds_df[["date", "match_id", "home_team_id"]]
    
    odds_df = odds_df.merge(bookprob, 
                            on=["date", "match_id", "home_team_id"],
                            how="left")

['B365H', 'B365D', 'B365A']
['BWH', 'BWD', 'BWA']
['IWH', 'IWD', 'IWA']
['LBH', 'LBD', 'LBA']
['PSH', 'PSD', 'PSA']
['WHH', 'WHD', 'WHA']
['SJH', 'SJD', 'SJA']
['VCH', 'VCD', 'VCA']
['GBH', 'GBD', 'GBA']
['BSH', 'BSD', 'BSA']


# Stack Match Result

In [10]:
match_df.groupby("match_id").size().value_counts()

1    26231
dtype: int64

In [11]:
match_df.columns

Index(['country_id', 'league_id', 'season', 'stage', 'date', 'match_id',
       'home_team_id', 'away_team_id', 'home_team_goal', 'away_team_goal',
       'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object')

In [12]:
starting_players_col = list(match_df.columns[10:32])

starting_players_df = match_df[["match_id"] + starting_players_col]

starting_players_df["total_missing"] = match_df[starting_players_col].isnull().sum(axis=1)

starting_players_df["none_missing"] = np.where(starting_players_df["total_missing"]==0,1,0)

starting_players_df["none_missing"].mean()

<ipython-input-12-39cd4820d52d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_players_df["total_missing"] = match_df[starting_players_col].isnull().sum(axis=1)
<ipython-input-12-39cd4820d52d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_players_df["none_missing"] = np.where(starting_players_df["total_missing"]==0,1,0)


0.8144180549731234

In [13]:
match_df["no_player_missing"] = starting_players_df["none_missing"]

match_df["no_player_missing"].mean()

0.8144180549731234

In [14]:
league_df

,league_id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [15]:
match_df.groupby("league_id")["no_player_missing"].mean()

league_id
1        0.615542
1729     0.974342
4769     0.942105
7809     0.970588
10257    0.903321
13274    0.831291
15722    0.244271
17642    0.623782
19694    0.844846
21518    0.890461
24558    0.828411
Name: no_player_missing, dtype: float64

In [16]:
match_df.groupby("season")["no_player_missing"].mean()

season
2008/2009    0.530066
2009/2010    0.762229
2010/2011    0.825767
2011/2012    0.863706
2012/2013    0.883436
2013/2014    0.848466
2014/2015    0.896272
2015/2016    0.907396
Name: no_player_missing, dtype: float64

In [17]:
match_df = match_df.loc[match_df["no_player_missing"]==1]

match_df = match_df.drop("no_player_missing", axis=1)

In [18]:
match_df["home_result"] = 0
match_df.loc[match_df["home_team_goal"] > match_df["away_team_goal"], "home_result"] = 1
match_df.loc[match_df["home_team_goal"] < match_df["away_team_goal"], "home_result"] = -1

match_df["away_result"] = 0
match_df.loc[match_df["home_team_goal"] < match_df["away_team_goal"], "away_result"] = 1
match_df.loc[match_df["home_team_goal"] > match_df["away_team_goal"], "away_result"] = -1

In [19]:
stacked_match_df = match_df[["date", "league_id", "match_id", "home_team_id", "home_result"]]

stacked_match_df["home"] = 1

stacked_match_df.columns = ["date", "league_id", "match_id", "team_id", "result", "home"]

print(stacked_match_df.shape)

temp = match_df[["date", "league_id", "match_id", "away_team_id", "away_result"]]

temp["home"] = 0

temp.columns = ["date", "league_id", "match_id", "team_id", "result", "home"]

print(temp.shape)

stacked_match_df = pd.concat([stacked_match_df, temp])

print(stacked_match_df.shape)

(21363, 6)
(21363, 6)
(42726, 6)


<ipython-input-19-27912321809a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stacked_match_df["home"] = 1
<ipython-input-19-27912321809a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["home"] = 0


In [20]:
stacked_match_df = stacked_match_df.sort_values(["date", "match_id", "home"])

In [21]:
player_by_match_df = player_by_match_df[["date"]+ player_by_match_df.select_dtypes([np.number]).columns.tolist()]

In [22]:
stacked_match_df.loc[stacked_match_df["match_id"]==665362]

,date,league_id,match_id,team_id,result,home
3716,2009-09-12,1,665362,8203,-1,0
3716,2009-09-12,1,665362,9985,1,1


In [23]:
len(player_by_match_df.loc[player_by_match_df["match_id"]==665362, "player_id"].unique())

21

In [24]:
print(stacked_match_df.shape)
stacked_match_df = stacked_match_df.merge(player_by_match_df, on=["date", "match_id", "team_id"])
print(stacked_match_df.shape)

(42726, 6)
(469985, 44)


In [25]:
stacked_match_df = stacked_match_df.loc[stacked_match_df["match_id"]!=665362]

In [26]:
stacked_match_df.groupby("match_id")["result"].mean().value_counts()

0    21362
Name: result, dtype: int64

In [27]:
stacked_match_df.groupby("match_id")["home"].mean().value_counts()

0.5    21362
Name: home, dtype: int64

In [28]:
feat_col = stacked_match_df.columns[7:]

feat_df = stacked_match_df[stacked_match_df.columns[:6]].drop_duplicates()

feat_col

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'dribbling', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'reactions', 'shot_power', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'penalties', 'marking',
       'standing_tackle', 'gk_diving', 'gk_handling', 'gk_kicking',
       'gk_positioning', 'gk_reflexes', 'attacking_work_rate_imp',
       'defensive_work_rate_imp', 'volleys_imp', 'curve_imp', 'agility_imp',
       'balance_imp', 'jumping_imp', 'vision_imp', 'sliding_tackle_imp'],
      dtype='object')

In [29]:
for col in feat_col:
    
    if "gk_" in col:
        
        max_df = (stacked_match_df.groupby(["date", "match_id", "team_id"])[col].max().reset_index())

        max_df.columns = ["date", "match_id", "team_id", "max_{}".format(col)]
        
        print(feat_df.shape)
        feat_df = feat_df.merge(max_df, on=["date", "match_id", "team_id"])
        print(feat_df.shape)
        
    else:
        avg_df = (stacked_match_df.groupby(["date", "match_id", "team_id"])[col].mean().reset_index())

        avg_df.columns = ["date", "match_id", "team_id", "avg_{}".format(col)]
    
        print(feat_df.shape)
        feat_df = feat_df.merge(avg_df, on=["date", "match_id", "team_id"])
        print(feat_df.shape)


(42724, 6)
(42724, 7)
(42724, 7)
(42724, 8)
(42724, 8)
(42724, 9)
(42724, 9)
(42724, 10)
(42724, 10)
(42724, 11)
(42724, 11)
(42724, 12)
(42724, 12)
(42724, 13)
(42724, 13)
(42724, 14)
(42724, 14)
(42724, 15)
(42724, 15)
(42724, 16)
(42724, 16)
(42724, 17)
(42724, 17)
(42724, 18)
(42724, 18)
(42724, 19)
(42724, 19)
(42724, 20)
(42724, 20)
(42724, 21)
(42724, 21)
(42724, 22)
(42724, 22)
(42724, 23)
(42724, 23)
(42724, 24)
(42724, 24)
(42724, 25)
(42724, 25)
(42724, 26)
(42724, 26)
(42724, 27)
(42724, 27)
(42724, 28)
(42724, 28)
(42724, 29)
(42724, 29)
(42724, 30)
(42724, 30)
(42724, 31)
(42724, 31)
(42724, 32)
(42724, 32)
(42724, 33)
(42724, 33)
(42724, 34)
(42724, 34)
(42724, 35)
(42724, 35)
(42724, 36)
(42724, 36)
(42724, 37)
(42724, 37)
(42724, 38)
(42724, 38)
(42724, 39)
(42724, 39)
(42724, 40)
(42724, 40)
(42724, 41)
(42724, 41)
(42724, 42)
(42724, 42)
(42724, 43)


In [30]:
feat_col = stacked_match_df.columns[7:]

for col in feat_col:
    
    if "gk_" not in col:
        top3avg_df = (stacked_match_df
                      .sort_values(col)
                      .groupby(["date", "match_id", "team_id"])
                      .tail(3)
                      .groupby(["date", "match_id", "team_id"])[col].mean()
                      .reset_index())

        top3avg_df.columns = ["date", "match_id", "team_id", "top3avg_{}".format(col)]

        print(feat_df.shape)
        feat_df = feat_df.merge(top3avg_df, on=["date", "match_id", "team_id"])
        print(feat_df.shape)

(42724, 43)
(42724, 44)
(42724, 44)
(42724, 45)
(42724, 45)
(42724, 46)
(42724, 46)
(42724, 47)
(42724, 47)
(42724, 48)
(42724, 48)
(42724, 49)
(42724, 49)
(42724, 50)
(42724, 50)
(42724, 51)
(42724, 51)
(42724, 52)
(42724, 52)
(42724, 53)
(42724, 53)
(42724, 54)
(42724, 54)
(42724, 55)
(42724, 55)
(42724, 56)
(42724, 56)
(42724, 57)
(42724, 57)
(42724, 58)
(42724, 58)
(42724, 59)
(42724, 59)
(42724, 60)
(42724, 60)
(42724, 61)
(42724, 61)
(42724, 62)
(42724, 62)
(42724, 63)
(42724, 63)
(42724, 64)
(42724, 64)
(42724, 65)
(42724, 65)
(42724, 66)
(42724, 66)
(42724, 67)
(42724, 67)
(42724, 68)
(42724, 68)
(42724, 69)
(42724, 69)
(42724, 70)
(42724, 70)
(42724, 71)
(42724, 71)
(42724, 72)
(42724, 72)
(42724, 73)
(42724, 73)
(42724, 74)
(42724, 74)
(42724, 75)


In [31]:
feat_df.columns

Index(['date', 'league_id', 'match_id', 'team_id', 'result', 'home',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy', 'avg_short_passing', 'avg_dribbling',
       'avg_free_kick_accuracy', 'avg_long_passing', 'avg_ball_control',
       'avg_acceleration', 'avg_sprint_speed', 'avg_reactions',
       'avg_shot_power', 'avg_stamina', 'avg_strength', 'avg_long_shots',
       'avg_aggression', 'avg_interceptions', 'avg_positioning',
       'avg_penalties', 'avg_marking', 'avg_standing_tackle', 'max_gk_diving',
       'max_gk_handling', 'max_gk_kicking', 'max_gk_positioning',
       'max_gk_reflexes', 'avg_attacking_work_rate_imp',
       'avg_defensive_work_rate_imp', 'avg_volleys_imp', 'avg_curve_imp',
       'avg_agility_imp', 'avg_balance_imp', 'avg_jumping_imp',
       'avg_vision_imp', 'avg_sliding_tackle_imp', 'top3avg_overall_rating',
       'top3avg_potential', 'top3avg_crossing', 'top3avg_finishing',
       'top3avg_headi

In [32]:
print(feat_df.shape)
feat_df = feat_df.merge(match_df[["season", "match_id"]].drop_duplicates(), on="match_id", how="left")
print(feat_df.shape)

(42724, 75)
(42724, 76)


In [33]:
feat_df.to_csv(dir+"feat_df.csv")

# Stacked matches

In [ ]:
feat_df = pd.read_csv(dir+"feat_df.csv")

feat_df = feat_df.drop("Unnamed: 0", axis=1)

feat_df.columns

In [ ]:
for l in feat_df["league_id"].unique():
    for s in feat_df["season"].unique():
        for col in feat_df.columns[6:-1]:
            
            mu = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].mean()
            sd = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].std()
            
            feat_df.loc[(feat_df["season"]==s) &
                        (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                      (feat_df["league_id"]==l), col] - mu
            
            feat_df.loc[(feat_df["season"]==s) &
                        (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                      (feat_df["league_id"]==l), col] / sd
        

In [ ]:
feat_df.groupby("season").size()

In [ ]:
feat_df.groupby("season")["match_id"].nunique()

In [ ]:
seasons = feat_df["season"].unique()

ctree_df = feat_df.loc[feat_df["season"].isin(seasons[0:4])]
ctree_df["fold"] = 1
ctree_df["test_set"] = 1
ctree_df.loc[ctree_df["season"].isin(seasons[0:3]), "test_set"] = 0

for i in range(1, len(seasons)-4):
    c_df = feat_df.loc[feat_df["season"].isin(seasons[i:i+4])]
    c_df["fold"] = i+1
    c_df["test_set"] = 1
    c_df.loc[c_df["season"].isin(seasons[i:i+3]), "test_set"] = 0
    
    ctree_df = pd.concat([ctree_df,c_df])

In [ ]:
ctree_df.groupby(["fold", "test_set"]).size()

In [ ]:
ctree_df = ctree_df[ctree_df.columns[4:]]

In [ ]:
ctree_df.columns

In [ ]:
ctree_df.to_csv(dir+"ctree_df.csv")

# Home matches/difference in attributes

In [21]:
feat_df = pd.read_csv(dir+"feat_df.csv")

feat_df = feat_df.drop("Unnamed: 0", axis=1)

feat_df.columns

Index(['date', 'league_id', 'match_id', 'team_id', 'result', 'home',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy', 'avg_short_passing', 'avg_dribbling',
       'avg_free_kick_accuracy', 'avg_long_passing', 'avg_ball_control',
       'avg_acceleration', 'avg_sprint_speed', 'avg_reactions',
       'avg_shot_power', 'avg_stamina', 'avg_strength', 'avg_long_shots',
       'avg_aggression', 'avg_interceptions', 'avg_positioning',
       'avg_penalties', 'avg_marking', 'avg_standing_tackle', 'max_gk_diving',
       'max_gk_handling', 'max_gk_kicking', 'max_gk_positioning',
       'max_gk_reflexes', 'avg_attacking_work_rate_imp',
       'avg_defensive_work_rate_imp', 'avg_volleys_imp', 'avg_curve_imp',
       'avg_agility_imp', 'avg_balance_imp', 'avg_jumping_imp',
       'avg_vision_imp', 'avg_sliding_tackle_imp', 'top3avg_overall_rating',
       'top3avg_potential', 'top3avg_crossing', 'top3avg_finishing',
       'top3avg_headi

In [22]:
feat_df = feat_df.sort_values(["match_id", "home"], ascending=[True, True])

In [23]:
feat_df.head()

,date,league_id,match_id,team_id,result,home,avg_overall_rating,avg_potential,avg_crossing,avg_finishing,...,top3avg_attacking_work_rate_imp,top3avg_defensive_work_rate_imp,top3avg_volleys_imp,top3avg_curve_imp,top3avg_agility_imp,top3avg_balance_imp,top3avg_jumping_imp,top3avg_vision_imp,top3avg_sliding_tackle_imp,season
0,2008-08-09,4769,483130,7819,-1,0,69.454545,74.727273,56.272727,53.454545,...,2.400000,2.800000,67.900000,70.666667,76.111111,75.400000,76.811111,76.333333,69.666667,2008/2009
1,2008-08-09,4769,483130,9827,1,1,74.454545,83.090909,53.090909,47.636364,...,2.666667,2.900000,76.333333,80.333333,83.666667,81.333333,88.000000,85.333333,83.000000,2008/2009
2,2008-08-09,4769,483131,9831,-1,0,68.818182,75.909091,50.818182,50.181818,...,2.388889,2.000000,71.333333,68.666667,74.666667,78.000000,75.888889,77.000000,72.333333,2008/2009
3,2008-08-09,4769,483131,9746,1,1,64.909091,72.454545,44.090909,43.636364,...,2.377778,2.411111,68.000000,61.333333,71.333333,71.666667,80.333333,71.000000,66.666667,2008/2009
16,2008-08-10,4769,483133,9941,-1,0,70.363636,78.000000,55.727273,45.454545,...,2.000000,3.000000,68.666667,77.666667,77.666667,82.000000,77.666667,79.333333,75.000000,2008/2009


In [24]:
for col in feat_df.columns[6:-1]:
    feat_df[col] = feat_df[col].diff()

In [25]:
feat_df = feat_df.loc[feat_df["home"]==1]

In [26]:
feat_df = feat_df.drop(["home"], axis=1)

In [34]:
(feat_df.groupby(["league_id"]).agg({"avg_overall_rating" : ["mean", "std"]}))
# .groupby("league_id")["avg_overall_rating"].mean())

avg_overall_rating          
                        mean       std
league_id                             
1                   0.035820  3.939701
1729                0.079553  5.339994
4769                0.154711  5.202056
7809                0.081841  4.876844
10257               0.096370  5.228207
13274              -0.033281  4.783394
15722              -0.184144  3.421616
17642              -0.052912  6.151427
19694               0.071913  5.068134
21518               0.044497  5.630449
24558               0.010573  4.853690

In [ ]:
for l in feat_df["league_id"].unique():
    for s in feat_df["season"].unique():
        for col in feat_df.columns[5:-1]:
            
            mu = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].mean()
            sd = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].std()
            
            feat_df.loc[(feat_df["season"]==s) &
                        (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                      (feat_df["league_id"]==l), col] - mu
            
            feat_df.loc[(feat_df["season"]==s) &
                        (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                      (feat_df["league_id"]==l), col] / sd
        

In [ ]:
feat_df.groupby("season").size()

In [ ]:
feat_df.groupby("season")["match_id"].nunique()

In [ ]:
seasons = feat_df["season"].unique()

ctree_df = feat_df.loc[feat_df["season"].isin(seasons[0:4])]
ctree_df["fold"] = 1
ctree_df["test_set"] = 1
ctree_df.loc[ctree_df["season"].isin(seasons[0:3]), "test_set"] = 0

for i in range(1, len(seasons)-4):
    c_df = feat_df.loc[feat_df["season"].isin(seasons[i:i+4])]
    c_df["fold"] = i+1
    c_df["test_set"] = 1
    c_df.loc[c_df["season"].isin(seasons[i:i+3]), "test_set"] = 0
    
    ctree_df = pd.concat([ctree_df,c_df])

In [ ]:
ctree_df.groupby(["fold", "test_set"]).size()

In [ ]:
ctree_df = ctree_df[ctree_df.columns[4:]]

In [ ]:
ctree_df.columns

In [ ]:
ctree_df.to_csv(dir+"ctree_player_diff_df.csv")

# Team attributes


In [3]:
feat_df = pd.read_csv(dir+"feat_df.csv")

feat_df = feat_df.drop("Unnamed: 0", axis=1)

feat_df.columns

Index(['date', 'league_id', 'match_id', 'team_id', 'result', 'home',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy', 'avg_short_passing', 'avg_dribbling',
       'avg_free_kick_accuracy', 'avg_long_passing', 'avg_ball_control',
       'avg_acceleration', 'avg_sprint_speed', 'avg_reactions',
       'avg_shot_power', 'avg_stamina', 'avg_strength', 'avg_long_shots',
       'avg_aggression', 'avg_interceptions', 'avg_positioning',
       'avg_penalties', 'avg_marking', 'avg_standing_tackle', 'max_gk_diving',
       'max_gk_handling', 'max_gk_kicking', 'max_gk_positioning',
       'max_gk_reflexes', 'avg_attacking_work_rate_imp',
       'avg_defensive_work_rate_imp', 'avg_volleys_imp', 'avg_curve_imp',
       'avg_agility_imp', 'avg_balance_imp', 'avg_jumping_imp',
       'avg_vision_imp', 'avg_sliding_tackle_imp', 'top3avg_overall_rating',
       'top3avg_potential', 'top3avg_crossing', 'top3avg_finishing',
       'top3avg_headi

In [4]:
team_att_df = pd.read_csv(dir+"team_att_df_imp.csv")

team_att_df = team_att_df.drop("Unnamed: 0", axis=1)

team_att_df.columns


Index(['team_id', 'date', 'defenceDefenderLineClass',
       'buildUpPlayPositioningClass', 'chanceCreationPositioningClass',
       'buildUpPlaySpeed', 'buildUpPlayPassing', 'chanceCreationPassing',
       'chanceCreationCrossing', 'chanceCreationShooting', 'defencePressure',
       'defenceAggression', 'defenceTeamWidth', 'buildUpPlayDribbling_imp',
       'missing_buildUpPlayDribbling'],
      dtype='object')

In [5]:
team_att_df["date_team_att"] = pd.to_datetime(team_att_df["date"])
feat_df["date"] = pd.to_datetime(feat_df["date"])

print(feat_df.shape)

feat_df = feat_df.merge(team_att_df.drop("date",axis=1), on="team_id", how="left")

feat_df["day_diff"] = (feat_df["date"] - feat_df["date_team_att"]).dt.days

feat_df = feat_df.loc[feat_df["day_diff"] >= 0]

feat_df = feat_df.sort_values(["team_id", "date", "day_diff"])

feat_df = feat_df.sort_values("day_diff").groupby(["team_id", "date"]).head(1)
feat_df.shape

(42724, 76)


(35057, 91)

In [6]:
for col in team_att_df.columns[2:-1]:
    if "Class" not in col:
        print(col)
        print(team_att_df.groupby("team_id")[col].std().mean())
        print("--------------------------------------------------")

buildUpPlaySpeed
8.914053051775703
--------------------------------------------------
buildUpPlayPassing
8.242744671840322
--------------------------------------------------
chanceCreationPassing
7.876966433328479
--------------------------------------------------
chanceCreationCrossing
8.136517765410979
--------------------------------------------------
chanceCreationShooting
8.076980654681046
--------------------------------------------------
defencePressure
7.66169519162797
--------------------------------------------------
defenceAggression
7.995128660669684
--------------------------------------------------
defenceTeamWidth
7.2979359159771295
--------------------------------------------------
buildUpPlayDribbling_imp
8.958549761478745
--------------------------------------------------
missing_buildUpPlayDribbling
0.46832218922872937
--------------------------------------------------


In [7]:
feat_df.columns

Index(['date', 'league_id', 'match_id', 'team_id', 'result', 'home',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy', 'avg_short_passing', 'avg_dribbling',
       'avg_free_kick_accuracy', 'avg_long_passing', 'avg_ball_control',
       'avg_acceleration', 'avg_sprint_speed', 'avg_reactions',
       'avg_shot_power', 'avg_stamina', 'avg_strength', 'avg_long_shots',
       'avg_aggression', 'avg_interceptions', 'avg_positioning',
       'avg_penalties', 'avg_marking', 'avg_standing_tackle', 'max_gk_diving',
       'max_gk_handling', 'max_gk_kicking', 'max_gk_positioning',
       'max_gk_reflexes', 'avg_attacking_work_rate_imp',
       'avg_defensive_work_rate_imp', 'avg_volleys_imp', 'avg_curve_imp',
       'avg_agility_imp', 'avg_balance_imp', 'avg_jumping_imp',
       'avg_vision_imp', 'avg_sliding_tackle_imp', 'top3avg_overall_rating',
       'top3avg_potential', 'top3avg_crossing', 'top3avg_finishing',
       'top3avg_headi

In [8]:
temp=feat_df.groupby("match_id")["team_id"].nunique().reset_index()

feat_df = feat_df.loc[feat_df["match_id"].isin(set(temp["match_id"]))]

In [9]:
feat_df = feat_df.drop(["day_diff", "date_team_att", "missing_buildUpPlayDribbling"], axis=1)

In [10]:
feat_df = feat_df.drop([c for c in feat_df.columns if "top3avg_gk" in c], axis=1)

In [11]:
feat_df.columns[:50]

Index(['date', 'league_id', 'match_id', 'team_id', 'result', 'home',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy', 'avg_short_passing', 'avg_dribbling',
       'avg_free_kick_accuracy', 'avg_long_passing', 'avg_ball_control',
       'avg_acceleration', 'avg_sprint_speed', 'avg_reactions',
       'avg_shot_power', 'avg_stamina', 'avg_strength', 'avg_long_shots',
       'avg_aggression', 'avg_interceptions', 'avg_positioning',
       'avg_penalties', 'avg_marking', 'avg_standing_tackle', 'max_gk_diving',
       'max_gk_handling', 'max_gk_kicking', 'max_gk_positioning',
       'max_gk_reflexes', 'avg_attacking_work_rate_imp',
       'avg_defensive_work_rate_imp', 'avg_volleys_imp', 'avg_curve_imp',
       'avg_agility_imp', 'avg_balance_imp', 'avg_jumping_imp',
       'avg_vision_imp', 'avg_sliding_tackle_imp', 'top3avg_overall_rating',
       'top3avg_potential', 'top3avg_crossing', 'top3avg_finishing',
       'top3avg_headi

In [12]:
feat_df.columns[50:]

Index(['top3avg_free_kick_accuracy', 'top3avg_long_passing',
       'top3avg_ball_control', 'top3avg_acceleration', 'top3avg_sprint_speed',
       'top3avg_reactions', 'top3avg_shot_power', 'top3avg_stamina',
       'top3avg_strength', 'top3avg_long_shots', 'top3avg_aggression',
       'top3avg_interceptions', 'top3avg_positioning', 'top3avg_penalties',
       'top3avg_marking', 'top3avg_standing_tackle',
       'top3avg_attacking_work_rate_imp', 'top3avg_defensive_work_rate_imp',
       'top3avg_volleys_imp', 'top3avg_curve_imp', 'top3avg_agility_imp',
       'top3avg_balance_imp', 'top3avg_jumping_imp', 'top3avg_vision_imp',
       'top3avg_sliding_tackle_imp', 'season', 'defenceDefenderLineClass',
       'buildUpPlayPositioningClass', 'chanceCreationPositioningClass',
       'buildUpPlaySpeed', 'buildUpPlayPassing', 'chanceCreationPassing',
       'chanceCreationCrossing', 'chanceCreationShooting', 'defencePressure',
       'defenceAggression', 'defenceTeamWidth', 'buildUpPlayDribbl

In [13]:
#for col in [c for c in feat_df.columns if "Class" in c]:
    
#    team_season = feat_df.groupby(["team_id", "season", col]).agg({"result": ["size", "sum"]}).reset_index()
#    team_season.columns = ["team_id", "season", col, "count01", "sum01"]

#    league_season = feat_df.groupby(["league_id", "season", col])["result"].mean().reset_index()
#    league_season.columns = ["league_id", "season", col, "avg"]

#    team_season_og = feat_df[["team_id", "league_id", "season", col]].drop_duplicates()

#    team_season_og = team_season_og.merge(team_season, on=["team_id", "season", col], how="left")

#    team_season_og = team_season_og.merge(league_season, on=["league_id", "season", col], how="left")

#    team_season_og["{}_smoothed_te".format(col)] = ((team_season_og["sum01"] + (4*team_season_og["avg"]) )/
#                                                    (team_season_og["count01"] + 4))
    
#    feat_df = feat_df.merge(team_season_og[["team_id", "season", "{}_smoothed_te".format(col), col]],
#                        on=["team_id", "season", col])
    
#    feat_df = feat_df.drop(col, axis=1)
    

In [14]:
feat_df.shape

(35057, 88)

In [15]:
feat_df = feat_df.sort_values(["match_id", "home"], ascending=[True, True])

In [16]:
numeric_cols = feat_df.select_dtypes([np.number]).columns
numeric_cols = [col for col in numeric_cols if col not in ["date", "team_id", 'result', 'league_id',
                                                           'match_id', "home"]]
numeric_cols

['avg_overall_rating',
 'avg_potential',
 'avg_crossing',
 'avg_finishing',
 'avg_heading_accuracy',
 'avg_short_passing',
 'avg_dribbling',
 'avg_free_kick_accuracy',
 'avg_long_passing',
 'avg_ball_control',
 'avg_acceleration',
 'avg_sprint_speed',
 'avg_reactions',
 'avg_shot_power',
 'avg_stamina',
 'avg_strength',
 'avg_long_shots',
 'avg_aggression',
 'avg_interceptions',
 'avg_positioning',
 'avg_penalties',
 'avg_marking',
 'avg_standing_tackle',
 'max_gk_diving',
 'max_gk_handling',
 'max_gk_kicking',
 'max_gk_positioning',
 'max_gk_reflexes',
 'avg_attacking_work_rate_imp',
 'avg_defensive_work_rate_imp',
 'avg_volleys_imp',
 'avg_curve_imp',
 'avg_agility_imp',
 'avg_balance_imp',
 'avg_jumping_imp',
 'avg_vision_imp',
 'avg_sliding_tackle_imp',
 'top3avg_overall_rating',
 'top3avg_potential',
 'top3avg_crossing',
 'top3avg_finishing',
 'top3avg_heading_accuracy',
 'top3avg_short_passing',
 'top3avg_dribbling',
 'top3avg_free_kick_accuracy',
 'top3avg_long_passing',
 'top3a

In [17]:
for col in [c for c in feat_df.columns if "Class" in c]:
    
    feat_df[col] = np.where(feat_df[col]==feat_df[col].value_counts().head(1).index[0],
                            2, 1)
    

In [18]:
feat_df[[c for c in feat_df.columns if "Class" in c]].head(10)

,defenceDefenderLineClass,buildUpPlayPositioningClass,chanceCreationPositioningClass
43852,2,2,2
43858,1,2,1
40408,2,2,2
40414,1,2,2
44634,1,2,2
44640,2,2,2
39317,2,2,2
39323,2,2,2
38945,2,2,2
38951,1,2,2


In [19]:
feat_df["buildUpPlayDribbling_imp"].isnull().sum()

0

In [20]:
feat_df["buildUpPlayDribbling_imp"] = feat_df["buildUpPlayDribbling_imp"].astype(float)

In [21]:
for col in numeric_cols + [c for c in feat_df.columns if "Class" in c]:
    feat_df[col] = feat_df[col].diff()

In [22]:
for col in [c for c in feat_df.columns if "Class" in c]:
    feat_df[col] = np.where(feat_df[col]==0, 0, 1)

In [23]:
feat_df = feat_df.loc[feat_df["home"]==1]

In [24]:
feat_df = feat_df.drop(["home"], axis=1)

In [25]:
feat_df.isnull().sum().tail()

chanceCreationShooting      0
defencePressure             0
defenceAggression           0
defenceTeamWidth            0
buildUpPlayDribbling_imp    0
dtype: int64

In [26]:
numeric_cols

['avg_overall_rating',
 'avg_potential',
 'avg_crossing',
 'avg_finishing',
 'avg_heading_accuracy',
 'avg_short_passing',
 'avg_dribbling',
 'avg_free_kick_accuracy',
 'avg_long_passing',
 'avg_ball_control',
 'avg_acceleration',
 'avg_sprint_speed',
 'avg_reactions',
 'avg_shot_power',
 'avg_stamina',
 'avg_strength',
 'avg_long_shots',
 'avg_aggression',
 'avg_interceptions',
 'avg_positioning',
 'avg_penalties',
 'avg_marking',
 'avg_standing_tackle',
 'max_gk_diving',
 'max_gk_handling',
 'max_gk_kicking',
 'max_gk_positioning',
 'max_gk_reflexes',
 'avg_attacking_work_rate_imp',
 'avg_defensive_work_rate_imp',
 'avg_volleys_imp',
 'avg_curve_imp',
 'avg_agility_imp',
 'avg_balance_imp',
 'avg_jumping_imp',
 'avg_vision_imp',
 'avg_sliding_tackle_imp',
 'top3avg_overall_rating',
 'top3avg_potential',
 'top3avg_crossing',
 'top3avg_finishing',
 'top3avg_heading_accuracy',
 'top3avg_short_passing',
 'top3avg_dribbling',
 'top3avg_free_kick_accuracy',
 'top3avg_long_passing',
 'top3a

In [27]:
for l in feat_df["league_id"].unique():
    for s in feat_df["season"].unique():
        for col in numeric_cols:
            
            if col != "buildUpPlayDribbling_imp":
            
                mu = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].mean()
                sd = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].std()

                feat_df.loc[(feat_df["season"]==s) &
                            (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                          (feat_df["league_id"]==l), col] - mu

                feat_df.loc[(feat_df["season"]==s) &
                            (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                          (feat_df["league_id"]==l), col] / sd
            
            elif col == "buildUpPlayDribbling_imp":
                
                mu = feat_df.loc[(feat_df["season"]==s) & (feat_df["league_id"]==l), col].mean()
                feat_df.loc[(feat_df["season"]==s) &
                        (feat_df["league_id"]==l), col] = feat_df.loc[(feat_df["season"]==s) &
                                                                      (feat_df["league_id"]==l), col] - mu

In [28]:
feat_df["buildUpPlayDribbling_imp"].isnull().sum()

0

In [29]:
feat_df.groupby("season").size()

season
2009/2010     820
2010/2011    2613
2011/2012    2743
2012/2013    2796
2013/2014    2685
2014/2015    2899
2015/2016    2964
dtype: int64

In [30]:
ctree_df = feat_df.reset_index(drop=True)

In [31]:
#import random

#test_size = int(len(feat_df)*0.2)
#for i in range(10):
#    test_matches = random.sample(feat_df["match_id"].tolist(), test_size)
#    feat_df["test_set_{}".format(i)] = 0
#    feat_df.loc[feat_df["match_id"].isin(test_matches), "test_set_{}".format(i)] = 1

In [32]:
feat_df.groupby("season").size()

season
2009/2010     820
2010/2011    2613
2011/2012    2743
2012/2013    2796
2013/2014    2685
2014/2015    2899
2015/2016    2964
dtype: int64

In [33]:
seasons = sorted(feat_df["season"].unique())

for i in range(1, len(seasons)-1):
    print(seasons[i], seasons[i+1])    
    feat_df["fold_1t_{}_test".format(i)] = "other"
    feat_df.loc[feat_df["season"]==seasons[i+1], "fold_1t_{}_test".format(i)] = "test"
    feat_df.loc[feat_df["season"]==seasons[i], "fold_1t_{}_test".format(i)] = "train"


2010/2011 2011/2012
2011/2012 2012/2013
2012/2013 2013/2014
2013/2014 2014/2015
2014/2015 2015/2016


In [34]:
i=5
feat_df.groupby("fold_1t_{}_test".format(i)).agg({"date": ["min", "max"]})

date           
                      min        max
fold_1t_5_test                      
other          2010-02-22 2014-05-18
test           2015-07-17 2016-05-25
train          2014-07-19 2015-05-31

In [35]:
for i in range(1, len(seasons)-2):

    print(seasons[i:i+3][:-1], seasons[i:i+3][-1])    
    feat_df["fold_2t_{}_test".format(i)] = "other"
    feat_df.loc[feat_df["season"]==seasons[i:i+3][-1], "fold_2t_{}_test".format(i)] = "test"
    feat_df.loc[feat_df["season"].isin(seasons[i:i+3][:-1]), "fold_2t_{}_test".format(i)] = "train"

['2010/2011', '2011/2012'] 2012/2013
['2011/2012', '2012/2013'] 2013/2014
['2012/2013', '2013/2014'] 2014/2015
['2013/2014', '2014/2015'] 2015/2016


In [36]:
feat_df.groupby("fold_2t_{}_test".format(i)).agg({"date": ["min", "max"]})

date           
                      min        max
fold_2t_4_test                      
other          2010-02-22 2013-06-01
test           2015-07-17 2016-05-25
train          2013-07-13 2015-05-31

In [37]:
for i in range(1, len(seasons)-3):
    print(seasons[i:i+4][:-1], seasons[i:i+4][-1])    
    feat_df["fold_3t_{}_test".format(i)] = "other"
    feat_df.loc[feat_df["season"]==seasons[i:i+4][-1], "fold_3t_{}_test".format(i)] = "test"
    feat_df.loc[feat_df["season"].isin(seasons[i:i+4][:-1]), "fold_3t_{}_test".format(i)] = "train"

['2010/2011', '2011/2012', '2012/2013'] 2013/2014
['2011/2012', '2012/2013', '2013/2014'] 2014/2015
['2012/2013', '2013/2014', '2014/2015'] 2015/2016


In [38]:
feat_df.groupby("fold_3t_{}_test".format(i)).agg({"date": ["min", "max"]})

date           
                      min        max
fold_3t_3_test                      
other          2010-02-22 2012-05-23
test           2015-07-17 2016-05-25
train          2012-07-13 2015-05-31

In [39]:
seasons = sorted(feat_df["season"].unique())

for i in range(1, len(seasons)-1):
    
    print(seasons[i], seasons[i+1])    
    
    current_set_dates = sorted(feat_df.loc[feat_df["season"]==seasons[i+1], "date"].unique())
    
    print(seasons[i], seasons[i+1])    
    feat_df["fold_1t_c_{}_test".format(i)] = "other"
    feat_df.loc[feat_df["season"]==seasons[i+1], "fold_1t_c_{}_test".format(i)] = "test"
    feat_df.loc[feat_df["season"]==seasons[i], "fold_1t_c_{}_test".format(i)] = "train"
    
    feat_df.loc[feat_df["date"].isin(current_set_dates[:int(len(current_set_dates)//2)]),
                "fold_1t_c_{}_test".format(i)] = "train"
    
    


2010/2011 2011/2012
2010/2011 2011/2012
2011/2012 2012/2013
2011/2012 2012/2013
2012/2013 2013/2014
2012/2013 2013/2014
2013/2014 2014/2015
2013/2014 2014/2015
2014/2015 2015/2016
2014/2015 2015/2016


In [40]:
feat_df.groupby("fold_1t_c_{}_test".format(i)).agg({"date": ["min", "max"]})

date           
                        min        max
fold_1t_c_5_test                      
other            2010-02-22 2014-05-18
test             2016-01-08 2016-05-25
train            2014-07-19 2016-01-06

In [41]:
feat_df.columns

Index(['date', 'league_id', 'match_id', 'team_id', 'result',
       'avg_overall_rating', 'avg_potential', 'avg_crossing', 'avg_finishing',
       'avg_heading_accuracy',
       ...
       'fold_2t_3_test', 'fold_2t_4_test', 'fold_3t_1_test', 'fold_3t_2_test',
       'fold_3t_3_test', 'fold_1t_c_1_test', 'fold_1t_c_2_test',
       'fold_1t_c_3_test', 'fold_1t_c_4_test', 'fold_1t_c_5_test'],
      dtype='object', length=104)

In [42]:
#seasons = feat_df["season"].unique()

#ctree_df = feat_df.loc[feat_df["season"].isin(seasons[0:4])]
#ctree_df["fold"] = 1
#ctree_df["test_set"] = 1
#ctree_df.loc[ctree_df["season"].isin(seasons[0:3]), "test_set"] = 0

#for i in range(1, len(seasons)-4):
#    c_df = feat_df.loc[feat_df["season"].isin(seasons[i:i+4])]
#    c_df["fold"] = i+1
#    c_df["test_set"] = 1
#    c_df.loc[c_df["season"].isin(seasons[i:i+3]), "test_set"] = 0
    
#    ctree_df = pd.concat([ctree_df,c_df])

In [43]:
feat_df = feat_df[feat_df.columns[4:].tolist() + ["league_id"]]

In [44]:
feat_df.isnull().sum()

result                0
avg_overall_rating    0
avg_potential         0
avg_crossing          0
avg_finishing         0
                     ..
fold_1t_c_2_test      0
fold_1t_c_3_test      0
fold_1t_c_4_test      0
fold_1t_c_5_test      0
league_id             0
Length: 101, dtype: int64

In [45]:
feat_df.to_csv(dir+"ctree_teamplayer_diff_df.csv")

In [46]:
feat_df.shape

(17520, 101)